Anlyzing tweetz for some sucessful brands

In [ ]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: textblob in e:\anaconda\lib\site-packages (0.18.0.post0)



In [4]:
pip install pandas 

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install numpy<2


Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.


In [15]:
pip install --upgrade pandas pyarrow textblob matplotlib


Note: you may need to restart the kernel to use updated packages.


In [14]:
pip list --outdated | findstr -v Package | ForEach-Object {pip install --upgrade $_.split()[0]}


Note: you may need to restart the kernel to use updated packages.


'ForEach-Object' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import re
from textblob import TextBlob
import matplotlib.pyplot as plt

def clean_text(text):
    # Remove special characters, URLs, and other unwanted elements
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)  # Remove special characters
    return text.strip()

def analyze_sentiment(tweets):
    positive = 0
    neutral = 0
    negative = 0

    for tweet in tweets:
        cleaned_tweet = clean_text(tweet)
        analysis = TextBlob(cleaned_tweet)
        sentiment_score = analysis.sentiment.polarity

        if sentiment_score > 0:
            positive += 1
        elif sentiment_score < 0:
            negative += 1
        else:
            neutral += 1

    total = len(tweets)
    positive_percentage = (positive / total) * 100
    neutral_percentage = (neutral / total) * 100
    negative_percentage = (negative / total) * 100

    return positive_percentage, neutral_percentage, negative_percentage

def plot_pie_chart(positive, neutral, negative):
    labels = 'Positive', 'Neutral', 'Negative'
    sizes = [positive, neutral, negative]
    colors = ['yellowgreen', 'lightskyblue', 'lightcoral']
    explode = (0.1, 0, 0)  # explode 1st slice

    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=140)
    plt.axis('equal')
    plt.title('Sentiment Analysis Results')
    plt.show()

def main():
    file_path = 'phone_tweets.csv'
    df = pd.read_csv(file_path, encoding='latin1')

    user_query = input("Enter the word to search in tweets: ")

    # Filter tweets containing the user-specified word
    filtered_tweets = df[df['tweet'].str.contains(user_query, case=False)]['tweet'].tolist()

    if not filtered_tweets:
        print(f"No tweets found containing the word '{user_query}'.")
    else:
        print("Filtered Tweets:")
        for tweet in filtered_tweets:
            print("-", tweet)

        positive, neutral, negative = analyze_sentiment(filtered_tweets)
        print(f"\nPositive: {positive:.2f}%")
        print(f"Neutral: {neutral:.2f}%")
        print(f"Negative: {negative:.2f}%")

        plot_pie_chart(positive, neutral, negative)

     

if __name__ == "__main__":
    main()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "e:\Anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "e:\Anaconda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "e:\Anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 205, in star

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

Extracting positive tweets of sucessfull brands for key factor analysis 

In [2]:

def extract_positive_tweets(dataset):
    positive_tweets = []
    
    for index, row in dataset.iterrows():
        text = row["tweet"]
        # Perform sentiment analysis using TextBlob
        analysis = TextBlob(text)
        # Consider tweets with a polarity score greater than 0.5 as positive
        if analysis.sentiment.polarity > 0.5:
            positive_tweets.append(text)
    
    return positive_tweets
dataset = pd.read_csv("phone_tweets.csv")
# Assuming your original dataset is already loaded into 'dataset' variable
positive_tweets = extract_positive_tweets(dataset)

# Create a new dataset called 'usethis' with the positive tweets
usethis = pd.DataFrame({"tweet": positive_tweets})

# Save the new dataset to a CSV file
usethis.to_csv("postive_allbrands.csv", index=False)


NameError: name 'pd' is not defined

key factor extraction 

In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
import spacy

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load English language model for spaCy
nlp = spacy.load("en_core_web_sm")

def extract_keywords(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    
    # Calculate word frequency distribution
    fdist = FreqDist(filtered_tokens)
    
    # Get the most common keywords
    keywords = [word for word, freq in fdist.most_common(50)]  # Adjust the number as needed
    
    return keywords

def extract_entities(text):
    doc = nlp(text)
    entities = [entity.text for entity in doc.ents if entity.label_ == "ORG" or entity.label_ == "PRODUCT"]
    return entities

def extract_positive_factors(tweets):
    positive_factors = defaultdict(int)
    
    for tweet in tweets:
        keywords = extract_keywords(tweet)
        entities = extract_entities(tweet)
        
        for keyword in keywords:
            positive_factors[keyword] += 1
        
        for entity in entities:
            positive_factors[entity] += 1
    
    # Sort the positive factors by frequency
    sorted_factors = sorted(positive_factors.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_factors

# Load your dataset
dataset = pd.read_csv("postive_allbrands.csv")  # Adjust the filename and path as needed

# Assuming your dataset has a column named "text" containing the tweets
tweets = dataset["tweet"].tolist()

# Use the extract_positive_factors function defined earlier
positive_factors_all = extract_positive_factors(tweets)





A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "e:\Anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "e:\Anaconda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "e:\Anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 205, in star

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

Analysing the tweets of a particular brand

In [4]:
import pandas as pd
import re
from textblob import TextBlob
import matplotlib.pyplot as plt

def clean_text(text):
    # Remove special characters, URLs, and other unwanted elements
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)  # Remove special characters
    return text.strip()

def analyze_sentiment(tweets):
    positive = 0
    neutral = 0
    negative = 0

    for tweet in tweets:
        cleaned_tweet = clean_text(tweet)
        analysis = TextBlob(cleaned_tweet)
        sentiment_score = analysis.sentiment.polarity

        if sentiment_score > 0:
            positive += 1
        elif sentiment_score < 0:
            negative += 1
        else:
            neutral += 1

    total = len(tweets)
    positive_percentage = (positive / total) * 100
    neutral_percentage = (neutral / total) * 100
    negative_percentage = (negative / total) * 100

    return positive_percentage, neutral_percentage, negative_percentage

def plot_pie_chart(positive, neutral, negative):
    labels = 'Positive', 'Neutral', 'Negative'
    sizes = [positive, neutral, negative]
    colors = ['yellowgreen', 'lightskyblue', 'lightcoral']
    explode = (0.1, 0, 0)  # explode 1st slice

    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=140)
    plt.axis('equal')
    plt.title('Sentiment Analysis Results')
    plt.show()

def main():
    file_path = 'Apple-Twitter-Sentiment-DFE.csv'
    df = pd.read_csv(file_path, encoding='latin1')

    user_query = input("Enter the word to search in tweets: ")

    # Filter tweets containing the user-specified word
    filtered_tweets = df[df['text'].str.contains(user_query, case=False)]['text'].tolist()

    if not filtered_tweets:
        print(f"No tweets found containing the word '{user_query}'.")
    else:
        print("Filtered Tweets:")
        for tweet in filtered_tweets:
            print("-", tweet)

        positive, neutral, negative = analyze_sentiment(filtered_tweets)
        print(f"\nPositive: {positive:.2f}%")
        print(f"Neutral: {neutral:.2f}%")
        print(f"Negative: {negative:.2f}%")

        plot_pie_chart(positive, neutral, negative)

     

if __name__ == "__main__":
    main()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "e:\Anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "e:\Anaconda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "e:\Anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 205, in star

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

Extarcting positive tweets of the particular brand

In [5]:
pip install TextBlob

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from textblob import TextBlob

# Load your dataset
def extract_positive_tweets(dataset):
    positive_tweets = []
    
    for index, row in dataset.iterrows():
        text = row["text"]
        # Perform sentiment analysis using TextBlob
        analysis = TextBlob(text)
        # Consider tweets with a polarity score greater than 0.5 as positive
        if analysis.sentiment.polarity > 0.5:
            positive_tweets.append(text)
    
    return positive_tweets
dataset = pd.read_csv("Apple-Twitter-Sentiment-DFE.csv",encoding="latin1")
# Assuming your original dataset is already loaded into 'dataset' variable
positive_tweets = extract_positive_tweets(dataset)

# Create a new dataset called 'usethis' with the positive tweets
usethis = pd.DataFrame({"tweet": positive_tweets})

# Save the new dataset to a CSV file
usethis.to_csv("usethis.csv", index=False)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "e:\Anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "e:\Anaconda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "e:\Anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 205, in star

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

Key factors that the brand contain

In [ ]:
pip install spacy

   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.8 MB 9.2 MB/s eta 0:00:02
   ------------- -------------------------- 3.9/11.8 MB 9.4 MB/s eta 0:00:01
   --------------------- ------------------ 6.3/11.8 MB 10.2 MB/s eta 0:00:01
   ----------------------------- ---------- 8.7/11.8 MB 10.5 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.8 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 11.8/11.8 MB 10.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 10.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ------ --------------------------------- 2.6/15.6 MB 13.8 MB/s eta 0:00:01
   ------------- -------------------------- 5.2/15.6 MB 13.3 MB/s eta 0:00:01
   -------------------- ------------------- 7.9/15.6 MB 12.5 MB/s eta 0:00:01
   ---------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
import spacy

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load English language model for spaCy
nlp = spacy.load("en_core_web_sm")

def extract_keywords(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    
    # Calculate word frequency distribution
    fdist = FreqDist(filtered_tokens)
    
    # Get the most common keywords
    keywords = [word for word, freq in fdist.most_common(50)]  # Adjust the number as needed
    
    return keywords

def extract_entities(text):
    doc = nlp(text)
    entities = [entity.text for entity in doc.ents if entity.label_ == "ORG" or entity.label_ == "PRODUCT"]
    return entities

def extract_positive_factors(tweets):
    positive_factors = defaultdict(int)
    
    for tweet in tweets:
        keywords = extract_keywords(tweet)
        entities = extract_entities(tweet)
        
        for keyword in keywords:
            positive_factors[keyword] += 1
        
        for entity in entities:
            positive_factors[entity] += 1
    
    # Sort the positive factors by frequency
    sorted_factors = sorted(positive_factors.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_factors

# Load your dataset
dataset = pd.read_csv("usethis.csv")  # Adjust the filename and path as needed

# Assuming your dataset has a column named "text" containing the tweets
tweets = dataset["tweet"].tolist()

# Use the extract_positive_factors function defined earlier
positive_factors = extract_positive_factors(tweets)

# Print the results
print("Key positive factors:")
for factor, count in positive_factors:
    print(f"{factor}: {count}")


ValueError: BLIS support requires blis: pip install blis

Extract key factors from positive tweets and store them in array for similarity calculation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
import spacy

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load English language model for spaCy
nlp = spacy.load("en_core_web_sm")

def extract_keywords(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    
    # Calculate word frequency distribution
    fdist = FreqDist(filtered_tokens)
    
    # Get the most common keywords
    keywords = [word for word, freq in fdist.most_common(50)]  # Adjust the number as needed
    
    return keywords

def extract_entities(text):
    doc = nlp(text)
    entities = [entity.text for entity in doc.ents if entity.label_ == "ORG" or entity.label_ == "PRODUCT"]
    return entities

def extract_positive_factors(tweets):
    positive_factors = defaultdict(int)
    
    for tweet in tweets:
        keywords = extract_keywords(tweet)
        entities = extract_entities(tweet)
        
        for keyword in keywords:
            positive_factors[keyword] += 1
        
        for entity in entities:
            positive_factors[entity] += 1
    
    # Sort the positive factors by frequency
    sorted_factors = sorted(positive_factors.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_factors

# Load data from positive_allbrands.csv
positive_allbrands_data = pd.read_csv("postive_allbrands.csv")  # Adjust the filename and path as needed

# Assuming your dataset has a column named "text" containing the tweets
positive_allbrands_tweets = positive_allbrands_data["tweet"].tolist()

# Extract positive factors from positive_allbrands.csv
positive_allbrands_factors = extract_positive_factors(positive_allbrands_tweets)

# Convert positive factors to a dictionary
positive_allbrands_dict = dict(positive_allbrands_factors)

# Load data from usethis.csv
usethis_data = pd.read_csv("usethis.csv")  # Adjust the filename and path as needed

# Assuming your dataset has a column named "text" containing the tweets
usethis_tweets = usethis_data["tweet"].tolist()

# Extract positive factors from usethis.csv
usethis_positive_factors = extract_positive_factors(usethis_tweets)

# Convert positive factors to a dictionary
usethis_positive_dict = dict(usethis_positive_factors)

# Combine positive factors of all brands and tweets
all_factors = list(positive_allbrands_dict.keys()) + list(usethis_positive_dict.keys())
all_factors = list(set(all_factors))  # Remove duplicates

# Create feature vectors for positive factors of all brands
positive_allbrands_vector = [positive_allbrands_dict.get(factor, 0) for factor in all_factors]

# Create feature vectors for positive factors of tweets in usethis.csv
usethis_positive_vector = [usethis_positive_dict.get(factor, 0) for factor in all_factors]

# Convert feature vectors to numpy arrays
positive_allbrands_array = np.array([positive_allbrands_vector])
usethis_positive_array = np.array([usethis_positive_vector])

# Calculate cosine similarity between positive factors of all brands and tweets in usethis.csv
cosine_similarity_scores = cosine_similarity(positive_allbrands_array, usethis_positive_array)

# Print the cosine similarity scores
print("Cosine Similarity Scores:", cosine_similarity_scores)


TypeError: ForwardRef._evaluate() missing 1 required keyword-only argument: 'recursive_guard'

## Based on the similarity score predict the brand success

In [ ]:
def predict_success_rate(similarity_scores, thresholds):
    success_rates = []
    for score in similarity_scores:
        if score >= thresholds["high"]:
            success_rates.append("High")
        elif score >= thresholds["moderate"]:
            success_rates.append("Moderate")
        else:
            success_rates.append("Low")
    return success_rates

# Define thresholds for success rate categories
thresholds = {
    "high": 0.8,
    "moderate": 0.4
}

# Use the similarity scores obtained earlier
similarity_scores = cosine_similarity_scores

# Predict success rates for the new brands
predicted_success_rates = predict_success_rate(similarity_scores, thresholds)

# Print the predicted success rates
print("Predicted Success Rates:", predicted_success_rates)


NameError: name 'cosine_similarity_scores' is not defined

In [17]:
pip install wordcloud Wordcloud

Note: you may need to restart the kernel to use updated packages.


In [18]:
#!/usr/bin/env python
# coding: utf-8

# In[6]:


# Import necessary libraries
#Nisha tyagi
get_ipython().system('pip install wordcloud Wordcloud')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveRegressor
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator



# In[7]:


# Load the data

data = pd.read_csv('instagram_data.csv', encoding='ISO-8859-1')
df = pd.DataFrame(data)
df


# In[8]:


# Check for missing values
data.isnull().sum()




# In[9]:


data.info()


# In[10]:


# Plot distribution of Impressions from different sources
plt.figure(figsize=(10, 8))
plt.style.use('fivethirtyeight')
plt.title("Distribution of Impressions From Home")
sns.distplot(data['From Home'])
plt.show()



# In[11]:


plt.figure(figsize=(10, 8))
plt.title("Distribution of Impressions From Hashtags")
sns.distplot(data['From Hashtags'])
plt.show()



# In[12]:


plt.figure(figsize=(10, 8))
plt.title("Distribution of Impressions From Explore")
sns.distplot(data['From Explore'])
plt.show()



# In[13]:


# Plot pie chart for Impressions from different sources using Plotly
home = data["From Home"].sum()
hashtags = data["From Hashtags"].sum()
explore = data["From Explore"].sum()
other = data["From Other"].sum()

labels = ['From Home', 'From Hashtags', 'From Explore', 'Other']
values = [home, hashtags, explore, other]

fig = px.pie(data, values=values, names=labels, title="Impressions on Instagram Posts From Various Sources",hole=0.5)
fig.show()



# In[14]:


# Generate and plot word cloud for captions and hashtags
text = " ".join(i for i in data.Caption)
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()



# In[15]:


text_hashtags = " ".join(i for i in data.Hashtags)
stopwords_hashtags = set(STOPWORDS)
wordcloud_hashtags = WordCloud(stopwords=stopwords_hashtags, background_color="white").generate(text_hashtags)
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud_hashtags, interpolation="bilinear")
plt.axis("off")
plt.show()



# In[16]:


# Scatter plots for relationships between variables using Plotly Express
fig = px.scatter(data_frame=data, x="Impressions", y="Likes", size="Likes", trendline="ols", title="Relationship Between Likes and Total Impressions")
fig.show()




# In[17]:


fig = px.scatter(data_frame=data, x="Impressions", y="Comments", size="Comments", trendline="ols", title="Relationship Between Comments and Total Impressions")
fig.show()



# In[18]:


fig = px.scatter(data_frame=data, x="Impressions", y="Shares", size="Shares", trendline="ols", title="Relationship Between Shares and Total Impressions")
fig.show()



# In[19]:


fig = px.scatter(data_frame=data, x="Impressions", y="Saves", size="Saves", trendline="ols", title="Relationship Between Post Saves and Total Impressions")
fig.show()



# In[20]:


# Calculate and print correlation values for different variables
correlation = data.corr()
print(correlation["Impressions"].sort_values(ascending=False))



# In[21]:


# Calculate and print conversion rate
conversion_rate = (data["Follows"].sum() / data["Profile Visits"].sum()) * 100
print(conversion_rate)



# In[22]:


# Scatter plot for relationship between Profile Visits and Followers Gained
fig = px.scatter(data_frame=data, x="Profile Visits", y="Follows", size="Follows", trendline="ols", title="Relationship Between Profile Visits and Followers Gained")
fig.show()



# In[23]:


# Train a PassiveAggressiveRegressor model
x = np.array(data[['Likes', 'Saves', 'Comments', 'Shares', 'Profile Visits', 'Follows']])
y = np.array(data["Impressions"])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

model = PassiveAggressiveRegressor()
model.fit(xtrain, ytrain)
score = model.score(xtest, ytest)
print(score)


# In[24]:


# Predict with the trained model
features = np.array([[282.0, 233.0, 4.0, 9.0, 165.0, 54.0]])
prediction = model.predict(features)
print(prediction)


# In[ ]:





# In[ ]:







A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "e:\Anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "e:\Anaconda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "e:\Anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 205, in star

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import